In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/pbp_related/"
img_DIR_P = "../data/images/players/"
data_DIR = "../data/pbpdata/"
box_DIR = "../data/box/"
export_DIR = "./fdata/"
source = "pbpstats"

In [ ]:
league = "NBA"
season = "2024"
year = season + "-" + str(int(season)+1)[-2:]
with zstd.open(data_DIR + league+"_PBPdata_" + season + ".pkl.zst","rb") as f:
    games_list = dill.load(f)

In [ ]:
box1 = pd.read_parquet(box_DIR + league + "_Box_T_Base_" + season + ".parquet")
box1 = pd.read_parquet(box_DIR + league + "_Box_T_Base_" + season + ".parquet")
box1.columns = map(str.lower,box1.columns)

In [ ]:
box1.query(f"game_id == '{id}'")

In [ ]:
box = pd.read_parquet(box_DIR + league + "_Box_T_Base_" + season + ".parquet")
box.columns = map(str.lower,box.columns)
box.loc[box.query(f"game_id == '0022400633' & team_id == 1610612759").index,"matchup"] = "SAS vs. IND"
box.loc[box.query(f"game_id == '0022400621' & team_id == 1610612754").index,"matchup"] = "IND vs. SAS"
box.loc[box.query(f"game_id == '0022400147' & team_id == 1610612764").index,"matchup"] = "WAS vs. MIA"
box.loc[box.query(f"game_id == '0022401229' & team_id == 1610612749").index,"matchup"] = "MIL vs. ATL"
box.loc[box.query(f"game_id == '0022401230' & team_id == 1610612760").index,"matchup"] = "OKC vs. HOU"
box.loc[box.query(f"game_id == '0022401229' & team_id == 1610612749").index,"matchup"] = "MIL vs. ATL"
box.loc[box.query(f"game_id == '0022401229' & team_id == 1610612749").index,"matchup"] = "MIL vs. ATL"


box = box.loc[box["matchup"].str.contains(" vs. ")]
# box["matchup"] = box["matchup"].str.replace(" ","")
box["matchup"] = box["matchup"].str.split(" vs. ",expand=True)[1]
box = box.rename(columns={"team_abbreviation":"home", "matchup":"away"})
box = box[["game_id","team_id","game_date","home","away"]]
box = box.reset_index(drop=True)

In [ ]:
# box = box[["GAME_ID","TEAM_ID"]].rename(columns={"GAME_ID":"game_id","TEAM_ID":"tidh"})
for game in games_list:
    id = game.game_id
    game.boxscore.team_home = box.query(f"game_id == '{id}'").iloc[0,1]
player_dict = get_players_pbp(league=league)  # Creating Player Dictionary
team_dict = nba_teams.get_teams()  # Creating Team Dictionary

In [ ]:
poss_list, points, poss_store = [], [], []
for game in tqdm(games_list):
    hmscr, rdscr = [], []
    tems = list(game.possessions.items[0].events[0].current_players.keys())
    tidh = game.boxscore.team_home
    for t in tems:
        if t!=tidh:
            game.boxscore.team_away = t
    game.boxscore.game_id = game.game_id
    game.boxscore.hpc = 0
    game.boxscore.apc = 0
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if possession_event.count_as_possession:
                off_id = possession_event.get_offense_team_id()
                if tidh == off_id:
                    game.boxscore.hpc+=1
                else:
                    game.boxscore.apc+=1

In [ ]:
boxscores = []
for game in tqdm(games_list):
    bb = game.boxscore
    boxscores.append([bb.game_id,bb.team_home,bb.team_away,bb.hpc,bb.apc])

In [ ]:
df1 =  pd.DataFrame(boxscores,columns=["game_id",'tid_home',"tid_away","poss_off_home","poss_def_home"])
df2 = pd.merge(box,df1)
df2.to_csv("../data/share/"+ "NBA_poss_counts_v2_2024.csv")